In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import re

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    minute  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour,minute)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)
    
def string2num2(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    return int(x)

PATH = "Jupyter Temp/lg 공모전/data/"

In [2]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_err_qual  = pd.read_csv(PATH+'train_quality_data.csv')
train_err_prob  = pd.read_csv(PATH+'train_problem_data.csv')

C:\Users\sue_b\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qual  = pd.read_csv(PATH+'test_quality_data.csv')

C:\Users\sue_b\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
model_test = pd.DataFrame(data={"user_id": test_err["user_id"].unique()})

In [8]:
model_test.sort_values(by="user_id",inplace=True)
model_test.head()

,user_id
0,30000
1,30001
2,30002
3,30003
4,30004


In [9]:
#모델 개수
model_test = model_test.merge(test_err.groupby(by="user_id")["model_nm"].nunique().reset_index(),how="left",on="user_id")
model_test.head()

,user_id,model_nm
0,30000,2
1,30001,1
2,30002,1
3,30003,1
4,30004,2


In [10]:
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [11]:
def model6(x):
    if x == "8.5.3":
        result = "08.5.3"
    elif x == "10":
        result = "10.00.00"
    else:
        result = x
    return result

In [12]:
test_err["fwver"] = test_err["fwver"].apply(lambda x : model6(x))
train_err["fwver"] = train_err["fwver"].apply(lambda x : model6(x))

In [13]:
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [14]:
temp = test_err.groupby(by=["user_id","model_nm"])["fwver"].nunique().reset_index()

In [15]:
temp["update"] = temp["fwver"].apply(lambda x : x-1)

In [16]:
temp = temp.groupby(by="user_id")["update"].sum().reset_index()

In [17]:
temp.head()

,user_id,update
0,30000,1
1,30001,0
2,30002,1
3,30003,1
4,30004,1


In [18]:
model_test = model_test.merge(temp,how="left",on="user_id")

In [19]:
model_test.head()

,user_id,model_nm,update
0,30000,2,1
1,30001,1,0
2,30002,1,1
3,30003,1,1
4,30004,2,1


In [32]:
display(test_err.head())
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [21]:
err_list = list(test_err["errtype"].unique())
err_list.sort()

In [22]:
len(err_list)

41

In [23]:
err_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [24]:
for i in range(1,43):
    model_test["err_"+str(i)] = [0] * len(model_test)

In [25]:
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,30000,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30002,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,30003,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,30004,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
set(test_qual["user_id"]) - set(test_err["user_id"])

{43262}

In [27]:
qual_only = pd.DataFrame(data=np.zeros((1,len(model_test.columns))), columns = model_test.columns)
qual_only.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
qual_only.iloc[0,0] = 43262
model_test = pd.concat([model_test,qual_only])
display(model_test.tail())
model_test.sort_values(by="user_id",inplace=True)
model_test.reset_index(drop=True,inplace=True)
display(model_test.tail())

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
14994,44995.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,44996.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,44997.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,44998.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,43262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
14994,44994.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14995,44995.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,44996.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,44997.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,44998.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
model_test["user_id"] = model_test["user_id"].apply(lambda x: int(x))
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,30000,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30002,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30003,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30004,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
id_error = test_err[['user_id','errtype']].values
error = np.zeros((test_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - test_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16532648/16532648 [01:37<00:00, 169745.81it/s]


(14999, 42)

In [34]:
error[:2]

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 6.200e+01, 1.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.600e+01, 1.600e+01,
        0.000e+00, 3.000e+00, 6.700e+01, 3.700e+01, 9.000e+00, 3.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 9.830e+02, 1.308e+03, 5.000e+00,
        2.000e+00, 3.200e+01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        8.300e+01, 2.000e+00, 1.600e+01, 0.000e+00, 1.000e+00, 3.000e+00,
        3.000e+00, 0.000e+00, 2.000e+00, 9.200e+01, 0.000e+00, 2.000e+00],
       [0.000e+00, 0.000e+00, 3.000e+00, 9.800e+01, 3.000e+00, 4.000e+00,
        4.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.200e+01, 1.600e+01,
        4.000e+00, 6.000e+00, 5.300e+01, 5.500e+01, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 2.400e+01, 1.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+

In [35]:
error_tp = error.transpose()
error_tp.shape

(42, 14999)

In [36]:
error_tp[:4]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  3.,  0., ...,  0.,  0.,  0.],
       [ 0., 98.,  0., ...,  0.,  1.,  2.]])

In [37]:
for i in range(len(error_tp)):
    model_test[model_test.columns[i+3]] = error_tp[i]   

In [38]:
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,30000,2.0,1.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0
1,30001,1.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30002,1.0,1.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0
3,30003,1.0,1.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0
4,30004,2.0,1.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0


In [39]:
del model_test["err_29"]

In [51]:
fw = test_err.groupby("model_nm")["fwver"].unique()

In [52]:
fw

model_nm
model_0    [04.22.1750, 04.22.1778, 04.22.1684, 04.22.147...
model_1     [04.16.3553, 04.16.3571, 04.16.3439, 04.16.3569]
model_2    [04.33.1261, 04.33.1149, 04.33.1185, 04.33.117...
model_3    [05.15.2138, 05.15.2120, 05.15.2092, 05.15.310...
model_4                 [03.11.1167, 03.11.1149, 03.11.1141]
model_5                 [04.82.1684, 04.82.1778, 04.82.1730]
model_6                                   [10.00.00, 08.5.3]
model_7                             [05.66.3237, 05.66.3571]
model_8     [04.73.2569, 04.73.2571, 04.73.2237, 04.73.2577]
Name: fwver, dtype: object

In [53]:
fw_dic = {}
for mod in fw:
    mod = mod.sort()

fw

model_nm
model_0    [04.22.1170, 04.22.1448, 04.22.1478, 04.22.160...
model_1     [04.16.3439, 04.16.3553, 04.16.3569, 04.16.3571]
model_2    [04.33.1125, 04.33.1149, 04.33.1171, 04.33.118...
model_3    [05.15.2092, 05.15.2114, 05.15.2120, 05.15.213...
model_4                 [03.11.1141, 03.11.1149, 03.11.1167]
model_5                 [04.82.1684, 04.82.1730, 04.82.1778]
model_6                                   [08.5.3, 10.00.00]
model_7                             [05.66.3237, 05.66.3571]
model_8     [04.73.2237, 04.73.2569, 04.73.2571, 04.73.2577]
Name: fwver, dtype: object

In [54]:
for i in fw:
    print(i)

['04.22.1170' '04.22.1448' '04.22.1478' '04.22.1608' '04.22.1656'
 '04.22.1666' '04.22.1684' '04.22.1750' '04.22.1772' '04.22.1778'
 '10.22.1770' '10.22.1780']
['04.16.3439' '04.16.3553' '04.16.3569' '04.16.3571']
['04.33.1125' '04.33.1149' '04.33.1171' '04.33.1185' '04.33.1261']
['05.15.2092' '05.15.2114' '05.15.2120' '05.15.2138' '05.15.3104']
['03.11.1141' '03.11.1149' '03.11.1167']
['04.82.1684' '04.82.1730' '04.82.1778']
['08.5.3' '10.00.00']
['05.66.3237' '05.66.3571']
['04.73.2237' '04.73.2569' '04.73.2571' '04.73.2577']


In [55]:
for i in fw:
    for j in range(len(i)):
        fw_dic[i[j]] = [j,len(i)]

In [56]:
fw_dic

{'04.22.1170': [0, 12],
 '04.22.1448': [1, 12],
 '04.22.1478': [2, 12],
 '04.22.1608': [3, 12],
 '04.22.1656': [4, 12],
 '04.22.1666': [5, 12],
 '04.22.1684': [6, 12],
 '04.22.1750': [7, 12],
 '04.22.1772': [8, 12],
 '04.22.1778': [9, 12],
 '10.22.1770': [10, 12],
 '10.22.1780': [11, 12],
 '04.16.3439': [0, 4],
 '04.16.3553': [1, 4],
 '04.16.3569': [2, 4],
 '04.16.3571': [3, 4],
 '04.33.1125': [0, 5],
 '04.33.1149': [1, 5],
 '04.33.1171': [2, 5],
 '04.33.1185': [3, 5],
 '04.33.1261': [4, 5],
 '05.15.2092': [0, 5],
 '05.15.2114': [1, 5],
 '05.15.2120': [2, 5],
 '05.15.2138': [3, 5],
 '05.15.3104': [4, 5],
 '03.11.1141': [0, 3],
 '03.11.1149': [1, 3],
 '03.11.1167': [2, 3],
 '04.82.1684': [0, 3],
 '04.82.1730': [1, 3],
 '04.82.1778': [2, 3],
 '08.5.3': [0, 2],
 '10.00.00': [1, 2],
 '05.66.3237': [0, 2],
 '05.66.3571': [1, 2],
 '04.73.2237': [0, 4],
 '04.73.2569': [1, 4],
 '04.73.2571': [2, 4],
 '04.73.2577': [3, 4]}

In [57]:
def fw_trans(x):
    return(float(fw_dic[x][0]) / (int(fw_dic[x][1]) - 1))

In [58]:
test_err["fw_rank"] = test_err["fwver"].apply(lambda x: fw_trans(x))
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,fw_rank
0,30000,20201101030227,model_1,04.16.3553,31,1,0.333333
1,30000,20201101030227,model_1,04.16.3553,33,2,0.333333
2,30000,20201101030228,model_1,04.16.3553,15,1,0.333333
3,30000,20201101030256,model_1,04.16.3553,22,1,0.333333
4,30000,20201101030300,model_1,04.16.3553,11,1,0.333333


In [59]:
fwrank_temp = test_err.groupby(by=["user_id","model_nm"])["fw_rank"].mean().reset_index()

In [60]:
fwrank_temp.head()

,user_id,model_nm,fw_rank
0,30000,model_1,0.424389
1,30000,model_2,1.000000
2,30001,model_3,0.750000
3,30002,model_0,0.676553
4,30003,model_0,0.697623


In [61]:
fwrank_temp = fwrank_temp.groupby(by="user_id")["fw_rank"].mean().reset_index()
fwrank_temp.head()

,user_id,fw_rank
0,30000,0.712195
1,30001,0.750000
2,30002,0.676553
3,30003,0.697623
4,30004,0.662306


In [62]:
len(fwrank_temp)

14998

In [63]:
model_test = model_test.merge(fwrank_temp,how="left",on="user_id")

In [64]:
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank
0,30000,2.0,1.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.712195
1,30001,1.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000
2,30002,1.0,1.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.676553
3,30003,1.0,1.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.697623
4,30004,2.0,1.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.662306


In [65]:
test_qual.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [66]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].apply(lambda x: string2num(x))
    test_qual[i] = test_qual[i].apply(lambda x: 0 if x == -1 else x)

In [67]:
test_qual.dtypes

time           int64
user_id        int64
fwver         object
quality_0      int64
quality_1      int64
quality_2      int64
quality_3      int64
quality_4      int64
quality_5      int64
quality_6      int64
quality_7      int64
quality_8      int64
quality_9      int64
quality_10     int64
quality_11     int64
quality_12     int64
dtype: object

In [68]:
test_qual_1 = test_qual.groupby(by=["time","user_id"]).mean().reset_index()

In [69]:
test_qual_1.head()

,time,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201031235000,30541,0.000000,0.000000,0.000000,0.0,0.0,0.416667,0.000000,0.0,0.0,0.0,5.0,0.000000,0.0
1,20201031235000,31139,0.000000,0.000000,0.000000,0.0,0.0,2.833333,0.000000,0.0,0.0,0.0,34.0,0.000000,0.0
2,20201031235000,36473,0.000000,0.000000,0.000000,0.0,0.0,9.250000,0.000000,0.0,0.0,0.0,111.0,0.000000,0.0
3,20201031235000,36482,6.666667,0.666667,6.666667,0.0,0.0,1.916667,0.666667,0.0,0.0,0.0,15.0,0.666667,0.0
4,20201031235000,36802,0.000000,0.000000,0.000000,0.0,0.0,0.083333,9.083333,109.0,0.0,0.0,1.0,0.000000,0.0


In [70]:
test_qual_max = test_qual_1.groupby(by="user_id").max().reset_index()
test_qual_max.head()

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0
1,30001,20201112040000,7.500000,0.750000,7.500000,0.0,0.0,213.000000,0.750000,0.0,0.0,0.0,2556.0,0.750000,0.0
2,30002,20201118143000,4.166667,0.416667,4.166667,0.0,0.0,4.083333,0.416667,0.0,0.0,0.0,49.0,0.416667,0.0
3,30003,20201125183000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,7.750000,93.0,0.0,0.0,6.0,0.000000,0.0
4,30004,20201126155000,5.000000,0.500000,5.000000,0.0,0.0,0.500000,2.500000,24.0,0.0,0.0,2.0,0.500000,0.0


In [71]:
for i in range(13):
    test_qual_max.rename(columns= {"quality_"+str(i) : "q_meanmax_"+str(i)},inplace=True)

test_qual_max.head()

,user_id,time,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12
0,30000,20201128195000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0
1,30001,20201112040000,7.500000,0.750000,7.500000,0.0,0.0,213.000000,0.750000,0.0,0.0,0.0,2556.0,0.750000,0.0
2,30002,20201118143000,4.166667,0.416667,4.166667,0.0,0.0,4.083333,0.416667,0.0,0.0,0.0,49.0,0.416667,0.0
3,30003,20201125183000,0.000000,0.000000,0.000000,0.0,0.0,0.500000,7.750000,93.0,0.0,0.0,6.0,0.000000,0.0
4,30004,20201126155000,5.000000,0.500000,5.000000,0.0,0.0,0.500000,2.500000,24.0,0.0,0.0,2.0,0.500000,0.0


In [72]:
len(test_qual_max)

8268

In [73]:
del test_qual_max["time"]

In [74]:
model_test = model_test.merge(test_qual_max,how="left",on="user_id")

In [75]:
model_test.isna().sum()

user_id            0
model_nm           0
update             0
err_1              0
err_2              0
err_3              0
err_4              0
err_5              0
err_6              0
err_7              0
err_8              0
err_9              0
err_10             0
err_11             0
err_12             0
err_13             0
err_14             0
err_15             0
err_16             0
err_17             0
err_18             0
err_19             0
err_20             0
err_21             0
err_22             0
err_23             0
err_24             0
err_25             0
err_26             0
err_27             0
err_28             0
err_30             0
err_31             0
err_32             0
err_33             0
err_34             0
err_35             0
err_36             0
err_37             0
err_38             0
err_39             0
err_40             0
err_41             0
err_42             0
fw_rank            1
q_meanmax_0     6731
q_meanmax_1     6731
q_meanmax_2  

In [76]:
model_test.fillna(0,inplace=True)

In [77]:
test_qual_2 = test_qual.groupby(by=["user_id","time"]).nunique().reset_index()
test_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,1,1,1,1,1,1,2,2,1,1,1,1,1,1
1,30001,20201102202000,1,2,2,2,1,1,3,2,1,1,1,1,2,1
2,30001,20201104202000,1,1,1,1,1,1,3,1,1,1,1,1,1,1
3,30001,20201112040000,1,1,1,1,1,1,4,1,1,1,1,1,1,1
4,30002,20201101000000,1,1,1,1,1,1,2,1,1,1,1,1,1,1


In [78]:
for i in test_qual_2.columns[3:]:
    test_qual_2[i] = test_qual_2[i].apply(lambda x: x-1)
test_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,1,0,0,0,0,0,1,1,0,0,0,0,0,0
1,30001,20201102202000,1,1,1,1,0,0,2,1,0,0,0,0,1,0
2,30001,20201104202000,1,0,0,0,0,0,2,0,0,0,0,0,0,0
3,30001,20201112040000,1,0,0,0,0,0,3,0,0,0,0,0,0,0
4,30002,20201101000000,1,0,0,0,0,0,1,0,0,0,0,0,0,0


In [79]:
test_qual_2 = test_qual_2.groupby(by="user_id").sum().reset_index()
test_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,1,0,0,0,0,0,1,1,0,0,0,0,0,0
1,30001,60603318444000,3,1,1,1,0,0,7,1,0,0,0,0,1,0
2,30002,202011080103000,10,3,3,3,0,0,16,3,0,0,0,0,3,0
3,30003,101005598621000,5,0,0,0,0,0,5,5,0,0,0,0,0,0
4,30004,40402247309000,2,1,1,1,0,0,2,3,0,0,0,0,1,0


In [80]:
for i in range(13):
    test_qual_2.rename(columns= {"quality_"+str(i) : "q_chg_"+str(i)},inplace=True)

test_qual_2.head()

,user_id,time,fwver,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_7,q_chg_8,q_chg_9,q_chg_10,q_chg_11,q_chg_12
0,30000,20201128195000,1,0,0,0,0,0,1,1,0,0,0,0,0,0
1,30001,60603318444000,3,1,1,1,0,0,7,1,0,0,0,0,1,0
2,30002,202011080103000,10,3,3,3,0,0,16,3,0,0,0,0,3,0
3,30003,101005598621000,5,0,0,0,0,0,5,5,0,0,0,0,0,0
4,30004,40402247309000,2,1,1,1,0,0,2,3,0,0,0,0,1,0


In [81]:
del test_qual_2["time"]
del test_qual_2["fwver"]

In [82]:
for i in [7,8,9,10,12]:
    del test_qual_2["q_chg_"+str(i)]

In [83]:
test_qual_2.head()

,user_id,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11
0,30000,0,0,0,0,0,1,1,0
1,30001,1,1,1,0,0,7,1,1
2,30002,3,3,3,0,0,16,3,3
3,30003,0,0,0,0,0,5,5,0
4,30004,1,1,1,0,0,2,3,1


In [84]:
model_test = model_test.merge(test_qual_2,how="left",on="user_id")

In [85]:
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11
0,30000,2.0,1.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.712195,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,30001,1.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,7.500000,0.750000,7.500000,0.0,0.0,213.000000,0.750000,0.0,0.0,0.0,2556.0,0.750000,0.0,1.0,1.0,1.0,0.0,0.0,7.0,1.0,1.0
2,30002,1.0,1.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.676553,4.166667,0.416667,4.166667,0.0,0.0,4.083333,0.416667,0.0,0.0,0.0,49.0,0.416667,0.0,3.0,3.0,3.0,0.0,0.0,16.0,3.0,3.0
3,30003,1.0,1.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.697623,0.000000,0.000000,0.000000,0.0,0.0,0.500000,7.750000,93.0,0.0,0.0,6.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
4,30004,2.0,1.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.662306,5.000000,0.500000,5.000000,0.0,0.0,0.500000,2.500000,24.0,0.0,0.0,2.0,0.500000,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0


In [86]:
model_test.shape

(14999, 66)

In [87]:
model_test.fillna(0,inplace=True)
model_test.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11
0,30000,2.0,1.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.712195,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,30001,1.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,7.500000,0.750000,7.500000,0.0,0.0,213.000000,0.750000,0.0,0.0,0.0,2556.0,0.750000,0.0,1.0,1.0,1.0,0.0,0.0,7.0,1.0,1.0
2,30002,1.0,1.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.676553,4.166667,0.416667,4.166667,0.0,0.0,4.083333,0.416667,0.0,0.0,0.0,49.0,0.416667,0.0,3.0,3.0,3.0,0.0,0.0,16.0,3.0,3.0
3,30003,1.0,1.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.697623,0.000000,0.000000,0.000000,0.0,0.0,0.500000,7.750000,93.0,0.0,0.0,6.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
4,30004,2.0,1.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.662306,5.000000,0.500000,5.000000,0.0,0.0,0.500000,2.500000,24.0,0.0,0.0,2.0,0.500000,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0


In [88]:
len(model_test)

14999

In [89]:
model_test.to_csv("Jupyter Temp/lg 공모전/data/0126_test_x.csv",index=False)